# How to cancel a running workflow

Let's say we have dispatched a workflow which is going to run for some time. During its execution, we realize that we have given some wrong input and thus want to cancel its execution. That can be done as follows,

In [1]:
import covalent as ct
import time

@ct.electron
def task_1():
    time.sleep(10)
    print("Task 1")
    return 5

@ct.electron
def task_2(b, c):
    time.sleep(b + c)
    print("Task 2")

@ct.lattice
def workflow(x):
    task_2(task_1(), x)
    return x ** 2

dispatch_id = ct.dispatch(workflow)(10)
print(dispatch_id)

bed2d751-478a-4d6d-af24-be01099ab65a


Now, when we try to get the result of this workflow within next 5 seconds,

In [2]:
time.sleep(3)
result = ct.get_result(dispatch_id)
print(result)


Lattice Result
status: RUNNING
result: None
inputs: {'x': 10}
error: None

start_time: 2022-01-23 01:00:55.892686+00:00
end_time: None

results_dir: /tmp/results
dispatch_id: bed2d751-478a-4d6d-af24-be01099ab65a

Node Outputs
------------
task_1(0): None
: None
:parameter:10(2): 10



We see that it hasn't finished executing yet. At this moment we realize that we want to cancel the execution of this workflow. We use the `ct.cancel` function,

In [3]:
ct.cancel_workflow(dispatch_id)

'Dispatch bed2d751-478a-4d6d-af24-be01099ab65a cancelled.'

Now we check the result again, and see outputs of all the nodes,

In [4]:
result = ct.get_result(dispatch_id, wait=True)
db = ct.get_data_store()
print(result)

print(result.get_all_node_outputs(db))


Lattice Result
status: CANCELLED
result: None
inputs: {'x': 10}
error: None

start_time: 2022-01-23 01:00:55.892686+00:00
end_time: 2022-01-23 01:01:01.065509+00:00

results_dir: /tmp/results
dispatch_id: bed2d751-478a-4d6d-af24-be01099ab65a

Node Outputs
------------
task_1(0): 5
task_2(1): None
:parameter:10(2): 10

{'task_1(0)': 5, 'task_2(1)': None, ':parameter:10(2)': 10}


We see that the after one of the nodes was executed, the cancellation signal was received by the dispatcher and further execution of the workflow was stopped. Since as of yet, `Dask` does not allow cancellation of a running thread, we cannot stop execution of a node that has already started running. So cancellation of a workflow execution will only affect the nodes which hadn't already started their execution and were going to do so after the cancellation signal was received.